In [ ]:
#Requires to have geopandas installed
#Installation https://geopandas.org/getting_started/install.html

import geopandas as gpd
#import geoplot
import pandas as pd
import fiona #;help(fiona.open)
import requests
from pandas.io.json import json_normalize # convert json into dataframe


In [ ]:
#For OSM 
import pyrosm
from pyrosm import OSM, get_data

In [ ]:
pd.set_option('display.max_rows', 100)
pd.set_option('display.max_columns', 100)

#  Adding MENA to Africa

In [ ]:
world = gpd.read_file(gpd.datasets.get_path('naturalearth_lowres'))
#world

In [ ]:
saudi = world.query('name == "Saudi Arabia"')
yemen = world.query('name == "Yemen"')
iraq = world.query('name == "Iraq"')
oman = world.query('name == "Oman"')
uae = world.query('name == "United Arab Emirates"')
jordan = world.query('name == "Jordan"')
palestine = world.query('name == "Palestine"')
israel = world.query('name == "Israel"')
qatar = world.query('name == "Qatar"')
kuwait = world.query('name == "Kuwait"')

mena = saudi.append(yemen).append(iraq).append(oman).append(uae).append(jordan).append(palestine).append(israel).append(qatar).append(kuwait)

In [ ]:
africa = world.query('continent == "Africa"').append(mena)
#africa.head()

In [ ]:
### Example plot https://geopandas.org/gallery/plotting_with_geoplot.html?highlight=africa

# #gdp_per_cap = world.gdp_md_est / world.pop_est
# ax = geoplot.cartogram(
#     africa, scale='pop_est', limits=(0.2, 1),
#     edgecolor='None', figsize=(7, 8)
# )
# geoplot.polyplot(africa, edgecolor='gray', ax=ax)

In [ ]:
#ax = africa.plot(color="white", edgecolors="red",figsize=(10, 10))

# 3) Substations - Onsset - OSM

Install PYROSM to handle OSM data: https://pyrosm.readthedocs.io/en/latest/installation.html

data source http://download.geofabrik.de/  

data description: https://gep-onsset.readthedocs.io/en/latest/GIS%20data%20collection.html

### OSM content check

In [ ]:
from pyrosm import OSM, get_data

# Initialize the OSM reader with test data
fp = get_data("Helsinki")
osm = OSM(fp)

# The instance has a configuration attribute containing:
print([item for item in osm.conf.__dict__.keys() if not item.startswith("_")])


In [ ]:
# Show all available tag attributes
osm.conf.tags.available


In [ ]:
# Show all tags that are converted into columns from building features
osm.conf.tags.power

## Create OSM substation dataframe

Pyrosm has an existing issue handling geofabric data. Thats why "get_data("africa")" is not working. 

A way around this problem is to download the data from http://download.geofabrik.de/africa-latest.osm.bz2 locally & set the path right

://github.com/HTenkanen/pyrosm/issues/109

In [ ]:
pwd

In [ ]:
cd

In [ ]:
cd "OneDrive/PHD-Flexibility/07_pypsa-africa/Data"

In [ ]:
africa_data = "africa-latest.osm.pbf"

In [ ]:
# Initialize the OSM object 
osm = pyrosm.OSM(africa_data)

In [ ]:
# By default pyrosm reads all elements having "amenity", "shop" or "tourism" tag
# Here, let's read only "amenity" and "shop" by applying a custom filter that
# overrides the default filtering mechanism
custom_filter = {'power': ["substation"]}
pois = osm.get_pois(custom_filter=custom_filter)

In [ ]:
# Gather info about POI type (combines the tag info from "amenity" and "shop")
pois["poi_type"] = pois["power"]

# Plot
ax = pois.plot(column='poi_type', markersize=3, figsize=(12,12), legend=True, legend_kwds=dict(loc='upper left', ncol=5, bbox_to_anchor=(1, 1))

In [ ]:
africa_substation = gpd.read_file(url)

In [ ]:
africa_substation.head()

In [ ]:
ax = africa_substation.plot(figsize=(20, 20),alpha = 0.7)
geoplot.polyplot(africa, edgecolor='gray', ax=ax)

# ADDITIONAL FOUND DATA. Network Data MENA (existing+planned)

In [ ]:
### datasource:   https://energydata.info/dataset/mena-electricity-transmission-network-2017


Main nodes and major lines of the electricity transmission network of the MENA region. Interconnection lines with countries outside of the region included. The properties for nodes are "name" and "node type" (city, town, plant, dam...). The properties for lines are the "transmission power" in kV (e.g. 400), the "type of line" (single, double...), the linked nodes (identified by the nodes names) and, for international interconnections, the "interconnection" property gathers the two linked countries identified by name. The format is GeoJSON, nodes are displayed as points and lines as lines. Data comes from the Arab Union of Electricity and country utilities.


In [ ]:
url = 'https://development-data-hub-s3-public.s3.amazonaws.com/ddhfiles/145365/electric-network-mena.geojson'
mena_network = gpd.read_file(url)

In [ ]:
# pd.set_option('display.max_rows', 600)
# pd.set_option('display.max_columns', 600)

In [ ]:
mena_network = mena_network.rename(columns={"transmissionPower": "voltage_kV"})
mena_network.head()

In [ ]:
ax = mena_network.plot(figsize=(20, 20),alpha = 0.7)
geoplot.polyplot(africa, edgecolor='gray', ax=ax)